In [ ]:
from imgPreprocessing import get_hpf
from keras.models import load_model
from predict_module import predict
from statistical_analysis.df_manipulation import gm_df_calcs, healthy_df_calcs, add_df_calcs, add_healthy_df_calcs

In [ ]:
in_path = input('What is the directory path to the image(s) you would like to analyse?\n')
out_path = input('What is the directory path you would like to save the results in?\n')
reuse = input('Are you adding more images from the same experiment?\n')
stat_path = 'content/drive/MyDrive/Stats/'
mod_path = '/content/drive/MyDrive/Models/'
entered_hpf = input('What stage of development were the provided images taken in?\n')
gm = input('What genetic modification has been applied to the embryo in the image(s)?\n')
scales = []
while scale != 'Done':
    scale = input('What is the height/width of the image in micrometres?\nIf all images have the same value, only enter it once.\n Enter Done when this has been completed for all images entered\n')
    scales.append(scale)

In [ ]:
# Get one of the default hpf values (30, 36, 48) based on which the entered value is closest to

mod_hpf = get_hpf(entered_hpf)

In [ ]:
# Define the required size parameters of the image for the model and the backbone used in the model

height = 512
width = 512
depth = 512
channels = 3
backbone = 'resnet34'

In [ ]:
# Load the model being used to make the predictions

model = load_model(mod_path+'{}HPF_AttentionResUnet_100epochs.h5'.format(mod_hpf), compile=True)

In [ ]:
# Import images, preprocess, make predictions and save the predicted masks

imgs, preds = predict(model, backbone, in_path, out_path)

In [ ]:
# Find volume of each class in the image
# Get the class labels for each stage of development

if mod_hpf == 30:
    classes = ['Background', 'Noise', 'Endocardium', 'Linear Heart Tube']
elif mod_hpf == 36:
    classes = ['Background', 'Noise', 'Endocardium', 'Atrium', 'Ventricle']
elif mod_hpf == 48:
    classes = ['Background', 'Noise', 'Endocardium', 'Atrium', 'AVC', 'Ventricle']

In [ ]:
# Complete calculations of the volumes of each class from the predicted masks and complete a statistical test as to whether the difference between
# the mean volume of each class of the entered images is statistically significantly different to that of the mean healthy volume of each class

# The results will be displayed in the terminal and saved as a CSV file in the user chose output path for access at a later date

if reuse == 'yes' or 'Yes' or 'YES' or 'y':
    if gm == 'healthy' or 'Healthy' or 'HEALTHY' or 'None' or 'none' or 'NONE':
        add_healthy_df_calcs(preds, classes, entered_hpf, scales, stat_path, out_path)
    else:
        add_df_calcs(preds, classes, entered_hpf, gm, scales, stat_path, out_path)

else:
    if gm == 'healthy' or 'Healthy' or 'HEALTHY' or 'None' or 'none' or 'NONE':
        add_healthy_df_calcs(preds, classes, entered_hpf, scales, stat_path, out_path)
    else:
        gm_df_calcs(preds, classes, entered_hpf, gm, scales, stat_path, out_path)